#### Ingest Constructors file into processes

##### Step 1 - Read JSON file using the spark dataframe reader

In [0]:
dbutils.widgets.text("p_data_source", "")
v_data_source = dbutils.widgets.get("p_data_source")

In [0]:
dbutils.widgets.text("p_file_date", "2021-03-21")
v_file_date = dbutils.widgets.get("p_file_date")

In [0]:
%run "../includes/configuration"

In [0]:
%run "../includes/common_functions"

In [0]:
constructors_schema = "constructorId INT, constructorRef STRING, name STRING, nationality STRING, url STRING"

In [0]:
constructor_df = spark.read \
.schema(constructors_schema) \
.json(f"{raw_folder_path}/{v_file_date}/constructors.json")

In [0]:
#constructor_df.printSchema()

In [0]:
# display(constructor_df)

##### Step 2 Dropping url column from df

In [0]:
constructor_drop_df = constructor_df.drop("url")
# constructor_drop_df = constructor_df.drop(constructor_df["url"]) ---- does the same thing


In [0]:
from pyspark.sql.functions import col, current_timestamp, lit

In [0]:
constructor_ingestion_date_df = add_ingestion_date(constructor_drop_df)

In [0]:
constructor_final_df = constructor_ingestion_date_df.withColumnRenamed("constructorId", "constructor_id") \
.withColumnRenamed("constructorRef", "constructor_ref") \
.withColumn("data_source", lit(v_data_source)) \
.withColumn("file_date", lit(v_file_date))

# display(constructor_final_df)

In [0]:
# constructor_final_df.write.mode("overwrite").parquet(f"{processed_folder_path}/constructors")

In [0]:
constructor_final_df.write.mode("overwrite").format("delta").saveAsTable("f1_processed.constructors")

In [0]:
%sql
SELECT * FROM f1_processed.constructors;

In [0]:
# display(spark.read.parquet(f"{processed_folder_path}/constructors"))

In [0]:
dbutils.notebook.exit("Success")